In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard 064365d8683fd002e9ad789c1e91fa3d021b44f0

Cloning into 'yolov5'...
remote: Enumerating objects: 17413, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 17413 (delta 64), reused 22 (delta 22), pack-reused 17327 (from 4)
Receiving objects: 100% (17413/17413), 16.29 MiB | 19.40 MiB/s, done.
Resolving deltas: 100% (11929/11929), done.
/content/yolov5/yolov5
HEAD is now at 064365d8 Update parse_opt() in export.py to work as in train.py (#10789)


In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 2.6.0+cu124 CPU


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="DGt4yQr48Cp4N8LJ4cR2")
project = rf.workspace("helmet-1ao1u").project("helmet-act4y")
version = project.version(1)
dataset = version.download("yolov5")



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Helmet-1 in yolov5pytorch:: 100%|██████████| 1270/1270 [00:00<00:00, 3752.42it/s]


In [ ]:
%cd /content/yolov5

/content/yolov5


In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

names:
- With Helmet
- Without Helmet
nc: 2
roboflow:
  license: CC BY 4.0
  project: helmet-act4y
  url: https://universe.roboflow.com/helmet-1ao1u/helmet-act4y/dataset/1
  version: 1
  workspace: helmet-1ao1u
test: ../test/images
train: Helmet-1/train/images
val: Helmet-1/valid/images


In [ ]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#this is the model configuration we will use for our tutorial
%cat /content/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
%cd /content/yolov5/
!sed -i "s/torch.load(f, map_location=torch.device('cpu'))/torch.load(f, map_location=torch.device('cpu'), weights_only=False)/" utils/general.py


/content/yolov5


In [ ]:
!python train.py --img 416 --batch 16 --epochs 100 --data {dataset.location}/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results --cache


2025-05-06 20:56:54.446830: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746565014.474210    4648 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746565014.486127    4648 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-06 20:56:54.515168: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter you

In [ ]:
print(dataset.location)


/content/yolov5/Helmet-1


In [ ]:
!ls /content/yolov5/Helmet-1/valid/images


BikesHelmets102_png.rf.7b1450aa30da62d3cca6e132c307f287.jpg
BikesHelmets106_png.rf.f0cefd4e3a16f6ed845bb5996bbc8fe6.jpg
BikesHelmets118_png.rf.ecf4b91cb8224a24289d4ecc9c4d7a77.jpg
BikesHelmets119_png.rf.cccffe1c38c813ca7709cea88b7a0837.jpg
BikesHelmets123_png.rf.361fadc396f37768c7aa3a24ebedfef7.jpg
BikesHelmets139_png.rf.eaf1f31a2c9864f83ec0333df73a7e68.jpg
BikesHelmets140_png.rf.4ed5f40b0249d0cd44e2258d5b125103.jpg
BikesHelmets142_png.rf.eac850235ed4698db415862f0283d71d.jpg
BikesHelmets14_png.rf.d4a9a77f8ce51d932c67afef199fb4c7.jpg
BikesHelmets150_png.rf.1d186b6987466ed4a14198e6628c8525.jpg
BikesHelmets151_png.rf.9e1a66b9a48b44f46742b9a346d5de68.jpg
BikesHelmets161_png.rf.e2dc679e6d47fdbd3c41e06d8b7c8756.jpg
BikesHelmets169_png.rf.7d0f71d5ece85c7a8abbee22a62c1847.jpg
BikesHelmets16_png.rf.367758b04a2d839420882ea70c5b1d62.jpg
BikesHelmets170_png.rf.ee6ffc0ce35fd938d7f5880119b7801d.jpg
BikesHelmets171_png.rf.416f44d210501edc6f2e178f81ceccf7.jpg
BikesHelmets17_png.rf.fceeabdfb77bad0ae7f8

In [ ]:
!cat /content/yolov5/Helmet-1/data.yaml


names:
- With Helmet
- Without Helmet
nc: 2
roboflow:
  license: CC BY 4.0
  project: helmet-act4y
  url: https://universe.roboflow.com/helmet-1ao1u/helmet-act4y/dataset/1
  version: 1
  workspace: helmet-1ao1u
test: ../test/images
train: Helmet-1/train/images
val: Helmet-1/valid/images


In [ ]:
data_yaml = """
train: /content/yolov5/Helmet-1/train/images
val: /content/yolov5/Helmet-1/valid/images
test: /content/yolov5/Helmet-1/test/images

nc: 2
names: ['With Helmet', 'Without Helmet']
"""

with open('/content/yolov5/Helmet-1/data.yaml', 'w') as f:
    f.write(data_yaml)


In [ ]:
!python train.py --img 416 --batch 16 --epochs 100 --data /content/yolov5/Helmet-1/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results --cache


2025-05-06 21:02:29.913187: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746565349.977366    6059 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746565349.993564    6059 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-06 21:02:30.048081: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter you

In [ ]:
import torch
import numpy.core
from models.yolo import DetectionModel
from torch.serialization import add_safe_globals

# نسمح لـ PyTorch بتحميل DetectionModel
add_safe_globals({'_reconstruct': numpy.core.multiarray._reconstruct})
add_safe_globals({'DetectionModel': DetectionModel})


In [ ]:
!find runs/train -name best.pt


In [ ]:
# تعديل دالة torch.load في YOLOv5
!sed -i "s/torch.load(attempt_download(w), map_location='cpu')/torch.load(attempt_download(w), map_location='cpu', weights_only=False)/" models/experimental.py


In [ ]:
!python detect.py --weights runs/train/yolov5s_results2/weights/best.pt --img 640 --conf 0.25 --source /content/BikesHelmets85.png


In [ ]:
from IPython.display import Image
Image(filename='runs/detect/exp5/BikesHelmets85.png')
